In [48]:
import pandas as pd
import plotly.io as pio
# Benutzerdefiniertes Template
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="Absolute Geschlechterverteilung von 2013 bis 2024",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

In [49]:
years = [str(year) for year in range(2013, 2025)]

df = pd.read_excel("data.xlsx", sheet_name=years)

data_list = []

for year, df in df.items():
    # Füge das Jahr als neue Spalte hinzu
    df['Jahr'] = year
    
    # Füge den DataFrame der Liste hinzu
    data_list.append(df)

# Alle DataFrames in der Liste zu einem einzigen DataFrame zusammenführen
final_df = pd.concat(data_list, ignore_index=True)


In [ ]:
df = final_df[final_df.columns[~final_df.columns.str.contains(r'\*')]]
df = df[df['Category'].isin(['Männlich (%)', 'Weiblich (%)', 'Anzahl Teilnahmen an Studierendenbefragung'])].fillna(value=0)
df = df.groupby(by=['Jahr', 'Category']).sum().reset_index()
df.columns = df.columns.str.replace('*', '', regex=False)


In [ ]:
absolute_zahlen = []
studiengaenge = df.columns[3:]

# Durch alle Jahre iterieren
for jahr in df["Jahr"].unique():
    df_jahr = df[df["Jahr"] == jahr]
    
    # Hole die Zeile mit der Anzahl der Studierenden
    anzahl_teilnahmen = df_jahr[df_jahr["Category"] == "Anzahl Teilnahmen an Studierendenbefragung"]
    
    # Hole die Zeilen mit den Prozentwerten für Männer und Frauen
    prozent_maenner = df_jahr[df_jahr["Category"] == "Männlich (%)"]
    prozent_frauen = df_jahr[df_jahr["Category"] == "Weiblich (%)"]
    
    # Berechnung der absoluten Zahlen
    if not anzahl_teilnahmen.empty and not prozent_maenner.empty and not prozent_frauen.empty:
        for studiengang in studiengaenge:
            anzahl = anzahl_teilnahmen[studiengang].values[0]
            
            # Zugriff auf den ersten Wert der Series und Berechnung
            maenner_abs = (anzahl * prozent_maenner[studiengang].iloc[0] / 100) if not pd.isna(prozent_maenner[studiengang].iloc[0]) else 0
            frauen_abs = (anzahl * prozent_frauen[studiengang].iloc[0] / 100) if not pd.isna(prozent_frauen[studiengang].iloc[0]) else 0
            
            absolute_zahlen.append([jahr, studiengang, int(maenner_abs), int(frauen_abs)])

# Umwandlung in DataFrame
df_absolute = pd.DataFrame(absolute_zahlen, columns=["Jahr", "Studiengang", "Männer", "Frauen"])


In [58]:
df = df_absolute.groupby('Jahr').sum().drop(columns='Studiengang')

In [80]:
df

,Männer,Frauen
Jahr,,
2013,117,473
2014,140,486
2015,99,437
2016,154,484
2017,146,503
2018,494,336
2019,194,669
2020,175,641
2021,187,658


In [81]:
import plotly.graph_objects as go
import plotly.express as px

# Angenommene Daten (df['Frauen'] und df['Männer'] sollten aus deinem DataFrame stammen)
women_bins = df['Frauen'] * -1  # Frauenpopulation mit negativen Werten
men_bins = df['Männer']  # Männerpopulation

# Altersgruppen (y-Achse)
y = list(range(2013, 2025, 1))

colors = px.colors.qualitative.Safe

# Layout des Diagramms
layout = go.Layout(
    title="Verteilung von </b><span style='color:#73c6e9;'>männlichen</span> <b> und </b><span style='color:#cc5b6e;'>weiblichen</span> <b> Studierenden im FB09",
    yaxis=go.layout.YAxis(title='Jahr'),
    xaxis=go.layout.XAxis(
        range=[-700, 700],
        tickvals=[-700, -525, -350, -175, 0, 175, 350, 525, 700],
        ticktext=[700, 525, 350, 175, 0, 175, 350, 525, 700],
        title='Anzahl Studierenden',
    ),
    
    barmode='overlay',  # Balken übereinander legen
    bargap=0.1
)

# Daten für die Balkendiagramme (Männer und Frauen)
data = [
    go.Bar(
        y=y,
        x=men_bins,
        orientation='h',
        name='Männer',
        text= men_bins.astype('int'),
        hoverinfo='x',
        marker=dict(color=colors[0]),
        
    ),
    go.Bar(
        y=y,
        x=women_bins,
        orientation='h',
        name='Frauen',
        text=-1 * women_bins.astype('int'),  # Text, um die positive Zahl anzuzeigen
        hoverinfo='text',
        marker=dict(color=colors[1]),
    )
]
fig = go.Figure(data=data, layout=layout)
fig.update_layout(template='plotly_white')  # Template nachträglich setzen


# Diagramm anzeigen
fig = go.Figure(data=data, layout=layout)
fig.update_layout(template='infoviz',showlegend=False)
fig.show()


In [82]:
import plotly.graph_objects as go
import plotly.express as px

# Angenommene Daten
years = list(range(2013, 2025, 1))  # X-Achse mit den Jahren
men_counts = df['Männer']  # Werte für Männer
women_counts = df['Frauen']  # Werte für Frauen

# Farbpalette von Plotly Express nutzen
colors = px.colors.qualitative.Safe

# Layout des Diagramms
layout = go.Layout(
    xaxis=dict(title='Jahr'),
    yaxis=dict(title='Anzahl Studierende'),
    barmode='group',
    template='infoviz',
    title="<b>Verteilung von </b><span style='color:#73c6e9;'>männlichen</span> <b> und </b><span style='color:#cc5b6e;'>weiblichen</span> <b> Studierenden im FB09",
    showlegend=False
)

# Daten für die Balkendiagramme
data = [
    go.Bar(
        x=years,
        y=men_counts,
        name='Männer',
        marker=dict(color=colors[0]),  # Erste Farbe aus der Safe-Palette
    ),
    go.Bar(
        x=years,
        y=women_counts,
        name='Frauen',
        marker=dict(color=colors[1]),  # Zweite Farbe aus der Safe-Palette
    )
]



# Diagramm anzeigen
fig = go.Figure(data=data, layout=layout)

fig.show()
